In [1]:
device= 'cuda:0'

data_technique = 'patchseq'
data_path = 'data/PatchSeq'
experiment = 'single_layer'

verbose = True

train_epoch = 50
learning_rate = 0.001
train_batch = 16
checkpoint = 1

#### Auto Programs and Parameters

In [2]:
import os

label_key = 'cell_type'
root_path = f'saved_patchseq/{experiment}'
os.makedirs(root_path, exist_ok=True)

log_path = f'{root_path}/log'
model_path = f'{root_path}/models'
tensorboard_path = f'{root_path}/tensorboards'
os.makedirs(model_path, exist_ok=True)
os.makedirs(tensorboard_path, exist_ok=True)

def print_time(name, start_time):
    print(f"{name}: {((time.time()-start_time)/60):0.2f} mins")

def clean_results(lp, mp, tp):
    if lp is not None and os.path.exists(lp):
        os.remove(lp)
    if mp is not None and os.path.exists(mp):
        shutil.rmtree(mp)
    if tp is not None and os.path.exists(tp):
        shutil.rmtree(tp)

#### Read and Process Data

In [3]:
from math import nan
import scipy.io as sio
import json
from sklearn import preprocessing
import pandas as pd
import numpy as np


alen_label = pd.read_excel(f'{data_path}/MET_data.xlsx')

Edat = 'E_pcipfx'
dir_pth = data_path
# Data operations:
D = sio.loadmat(dir_pth + '/PS_v5_beta_0-4_pc_scaled_ipfx_eqTE.mat', squeeze_me=True)
with open(dir_pth + '/E_names.json') as f:
    ephys_names = json.load(f)
D[Edat] = np.concatenate([D['E_pc_scaled'], D['E_feature']], axis=1)
D['pcipfx_names'] = np.concatenate([D['pc_name'], D['feature_name']])
temp = [ephys_names[f] for f in D['pcipfx_names']]
D['pcipfx_names'] = np.array(temp)
good_idx = ~np.isnan(D[Edat]).any(axis=1)
E_dat = D[Edat][good_idx]
T_dat = D['T_dat'][good_idx]

id_label = D['E_spec_id_label'][good_idx]


morph_feature = pd.read_csv(dir_pth + '/morph_features_gouwens.csv', index_col=0)
M_dat = morph_feature.iloc[:,1:].values
cell_id = morph_feature['cell_id']

cell_id,x_ind,_ = np.intersect1d(cell_id, alen_label['Specimen ID'], return_indices=True)
M_dat = M_dat[x_ind]

xy, x_ind, y_ind = np.intersect1d(id_label, cell_id, return_indices=True)
E_dat = E_dat[x_ind]
T_dat = T_dat[x_ind]
cell_type_MET = pd.Series(cell_id[y_ind]).map(dict(zip(alen_label['Specimen ID'],alen_label['MET type'])))
cell_type_T = pd.Series(cell_id[y_ind]).map(dict(zip(D['T_spec_id_label'],D['cluster'])))
cell_type = cell_type_MET
M_dat = M_dat[y_ind]



# cell_type_less = [ct.split('-')[0] for ct in cell_type.values]

scaler_T = preprocessing.StandardScaler().fit(T_dat)
T_dat = scaler_T.transform(T_dat)
scaler_E = preprocessing.StandardScaler().fit(E_dat)
E_dat = scaler_E.transform(E_dat)
scaler_M = preprocessing.StandardScaler().fit(M_dat)
M_dat = scaler_M.transform(M_dat)

cell_type_T_less = np.array([_ls.split(' ')[0] for _ls in cell_type_T])

E_dat = E_dat[cell_type_T_less!='Serpinf1',:]
M_dat = M_dat[cell_type_T_less!='Serpinf1',:]

cell_type_T_less = cell_type_T_less[cell_type_T_less!='Serpinf1']

import anndata as ad
adataE = ad.AnnData(E_dat)
adataE.obs[label_key] = cell_type_T_less
adataM = ad.AnnData(M_dat) 
adataM.obs[label_key] = cell_type_T_less
adatas_train = [adataE, adataM]


C:\ProgramData\Anaconda3\envs\untest\lib\site-packages\ipykernel_launcher.py:62: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
C:\ProgramData\Anaconda3\envs\untest\lib\site-packages\ipykernel_launcher.py:64: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.


In [4]:
adatas_train[0].obs[label_key]

0        Vip
1        Sst
2        Sst
3        Sst
4        Sst
       ...  
442    Lamp5
443    Lamp5
444      Vip
445      Vip
446    Pvalb
Name: cell_type, Length: 447, dtype: object

#### Train Model

In [5]:
from src import UnitedNet
import anndata as ad
import os, shutil
import time 

start_time = time.time()

model = UnitedNet(
    device=device,
    log_path=log_path,
    model_path=model_path,
    tensorboard_path=tensorboard_path,
    verbose=verbose,
)

# ======================================== Register Data ========================================
checkpoint_time = time.time()
model.register_anndatas(
    adatas_train, 
    label_index=0, label_key=label_key,
    technique=data_technique,
)
print_time('Register data', checkpoint_time)
print(model.model)

# ======================================== Train ========================================
checkpoint_time = time.time()
clean_results(log_path, model_path, tensorboard_path)
model.train(
    'unsupervised_group_identification', n_epoch=train_epoch, learning_rate=learning_rate, batch_size=train_batch,
    save_best_model=True, checkpoint=checkpoint,
)

print_time(f'Total', start_time)

Register data: 0.00 mins
Model(
  (modules_by_names): ModuleDict(
    (encoders): ModuleList(
      (0): MLP(
        (layers): ModuleList(
          (0): Linear(in_features=68, out_features=1024, bias=True)
          (1): ReLUActivation()
          (2): Linear(in_features=1024, out_features=512, bias=True)
          (3): ReLUActivation()
          (4): Linear(in_features=512, out_features=256, bias=True)
          (5): ReLUActivation()
          (6): Linear(in_features=256, out_features=128, bias=True)
          (7): ReLUActivation()
          (8): Linear(in_features=128, out_features=68, bias=True)
          (9): LayerNorm((68,), eps=1e-05, elementwise_affine=True)
        )
      )
      (1): MLP(
        (layers): ModuleList(
          (0): Linear(in_features=514, out_features=1024, bias=True)
          (1): Dropout(p=0.1, inplace=False)
          (2): ReLUActivation()
          (3): Linear(in_features=1024, out_features=512, bias=True)
          (4): ReLUActivation()
          (5)

In [10]:
for epoch in range(1, train_epoch+1):
    model.load_model(f'saved_patchseq/single_layer/models/train_0_translation/epoch_{epoch}.pt')
    model.set_verbose(False)
    print(epoch, model.evaluate()['ari'])

1 0.0
2 0.6663029288478295
3 0.7093068470052806
4 0.492362100506377
5 0.4833087267638936
6 0.3557739055343197
7 0.36423060955610614
8 0.38321729325808573
9 0.41302156063368023
10 0.43316706376327285
11 0.48745742668747877
12 0.43592010869515335
13 0.37779099215061
14 0.49028137608660355
15 0.46142612598932486
16 0.38171760914847475
17 0.4702770447382624
18 0.3736523273438997
19 0.39713074479016963
20 0.45103669318249706
21 0.4854492894289575
22 0.43677492724580036
23 0.42911738250301495
24 0.3588399861550133
25 0.4878582670483996
26 0.44925177255628107
27 0.4844368807405012
28 0.3591363151511391
29 0.35584063029907426
30 0.46812692876601186
31 0.4511836554359859
32 0.4865115922545949
33 0.4709573383188222
34 0.47443175911487706
35 0.4432946335152109
36 0.48352176309177847
37 0.46478361106638283
38 0.45042557615087614
39 0.4613856210913737
40 0.3858097982963604
41 0.3559791358864154
42 0.44769471265370503
43 0.42830903084030386
44 0.4402653239908076
45 0.44490046011790196
46 0.440192460

In [7]:
model.model.config

ModelConfig(input_sizes=[68, 514], output_size=5, class_weights=[2.0318181818181817, 1.0642857142857143, 4.966666666666667, 0.39210526315789473, 1.2246575342465753], encoders=[MLPConfig(input_size=68, output_size=68, hidden_sizes=[1024, 512, 256, 128], is_binary_input=False, activations=[ActivationConfig(method='relu'), ActivationConfig(method='relu'), ActivationConfig(method='relu'), ActivationConfig(method='relu'), None], dropouts=0.0, use_biases=True, use_batch_norms=False, use_layer_norms=[False, False, False, False, True]), MLPConfig(input_size=514, output_size=68, hidden_sizes=[1024, 512, 256, 128], is_binary_input=False, activations=[ActivationConfig(method='relu'), ActivationConfig(method='relu'), ActivationConfig(method='relu'), ActivationConfig(method='relu'), None], dropouts=[0.1, 0.0, 0.0, 0.0, 0.0], use_biases=True, use_batch_norms=False, use_layer_norms=[False, False, False, False, True])], decoders=[MLPConfig(input_size=68, output_size=68, hidden_sizes=[128, 256, 512, 10

#### Evaluate

In [8]:
import scanpy as sc
from src import UnitedNet
def evaluate_adatas(path, adatas):
    model = UnitedNet(
        device=device,
        log_path=None,
        model_path=None,
        tensorboard_path=None,
        verbose=False,
    )
    model.load_model(path)

    return model.evaluate(
        adatas,
        label_index_evaluate=0, label_key_evaluate=label_key,
        batch_index_evaluate=0, batch_key_evaluate=use_batch_key,
    )['ari']

In [9]:
final_model_path = f'{root_path}/models/{test_batch}/transfer_{final_step}_classification'
aris = []
for epoch in range(1, train_epoch+1):
    adatas_train, adatas_test = split_data(test_batch)
    adatas_all = concat_adatas(adatas_train, adatas_test)

    model_path = f'{final_model_path}/epoch_{epoch}.pt'
    aris.append(evaluate_adatas(model_path, adatas_test))

for epoch in range(1, train_epoch+1):
    print('epoch', epoch, aris[epoch-1])

NameError: name 'test_batch' is not defined

#### Infer

##### Infer Commons

In [ ]:
import scanpy as sc
from src import UnitedNet
def infer_adatas(path, adatas, eval_only=True):
    model = UnitedNet(
        device=device,
        log_path=None,
        model_path=None,
        tensorboard_path=None,
        verbose=True,
    )
    model.load_model(path)

    model.evaluate(
        adatas,
        label_index_evaluate=0, label_key_evaluate=label_key,
        batch_index_evaluate=0, batch_key_evaluate=use_batch_key,
    )

    if not eval_only:
      adata_inferred = model.infer(
          adatas,
          modalities_provided=list(range(len(adatas))),
          batch_index_infer=0, batch_key_infer=use_batch_key, 
          modality_sizes=[adata.shape[1] for adata in adatas]
      )
      
      adata_inferred.obs[modified_batch_key] = list(adatas[0].obs[modified_batch_key])
      sc.pl.umap(adata_inferred, color=[modified_batch_key])

      adata_inferred.obs['batch'] = list(adatas[0].obs['batch'])
      sc.pl.umap(adata_inferred, color=['batch'])
      
      sc.pl.umap(adata_inferred, color=['predicted_label'])
      
      adata_inferred.obs[label_key] = list(adatas[0].obs[label_key])
      sc.pl.umap(adata_inferred, color=[label_key])

##### Infer on train model

In [ ]:
final_model_path = f'{root_path}/models/{test_batch}/transfer_{final_step}_classification'
for epoch in range(1, train_epoch+1):
    print('='*20, 'epoch', epoch)

    adatas_train, adatas_test = split_data(test_batch)
    adatas_all = concat_adatas(adatas_train, adatas_test)

    model_path = f'{final_model_path}/epoch_{epoch}.pt'
    infer_adatas(model_path, adatas_test)
    infer_adatas(model_path, adatas_all, eval_only=False)
    

In [ ]:
final_model_path = f'{root_path}/models/{test_batch}/transfer_{final_step}_classification'
for epoch in [train_epoch]:
    print('='*20, 'epoch', epoch)

    adatas_train, adatas_test = split_data(test_batch)
    adatas_all = concat_adatas(adatas_train, adatas_test)

    model_path = f'{final_model_path}/epoch_{epoch}.pt'
    infer_adatas(model_path, adatas_test)
    infer_adatas(model_path, adatas_all, eval_only=False)
    